<a href="https://colab.research.google.com/github/shreeyut1905/AI-Optimizer/blob/main/AI_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements

In [ ]:
%pip install -U --quiet langchain langchain-chroma langchain-community openai langchain-experimental
%pip install --quiet "unstructured[all-docs]" pypdf pillow pydantic lxml pillow matplotlib chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.7 MB/s eta 0:

In [ ]:
!pip install langchain_core langchain langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 963.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00


## Graph

In [ ]:
from typing import Annotated , Any , Dict , Sequence , TypedDict
import operator
from langchain_core.messages import BaseMessage


import json

def merge_dicts(a:Dict[str,Any],b:Dict[str,Any]) -> Dict[str,Any]:
  return {**a,**b}


class AgentState(TypedDict):
  messages:Annotated[Sequence[BaseMessage],operator.add]
  date: Annotated[Dict[str,Any],merge_dicts]
  metadata:Annotated[Dict[str,Any],merge_dicts]


def show_agent_reasoning(output,agent_name):
  print(f"\n{'=' * 10} {agent_name.center(28)} {'=' * 10}")


  def convert_to_serializable(obj):
      if hasattr(obj, 'to_dict'):  # Handle Pandas Series/DataFrame
          return obj.to_dict()
      elif hasattr(obj, '__dict__'):  # Handle custom objects
          return obj.__dict__
      elif isinstance(obj, (int, float, bool, str)):
          return obj
      elif isinstance(obj, (list, tuple)):
          return [convert_to_serializable(item) for item in obj]
      elif isinstance(obj, dict):
          return {key: convert_to_serializable(value) for key, value in obj.items()}
      else:
          return str(obj)  # Fallback to string representation
  if isintance(output,(dict,list)):
    serializable_output = convert_to_serializable(output)
    print(json.dumps(serializable_output,indent=2))
  else:
    try:
      parsed_output = json.loads(output)
      print(json.dumps(parsed_output,indent=2))
    except json.JSONDecodeError:
      print(output)
  print("="*48)


## Tools

In [ ]:
import os
from typing import Dict, Any, List
import pandas as pd
import requests

In [ ]:
def get_financial_metrics(ticker:str,report_period:str,period:str = "ttm" , limit = 1) -> List[Dict[str,Any]]:
  headers = {"X-API-KEY": os.environ.get("FINANCIAL_DATASETS_API_KEY")}
  url = (
        f"https://api.financialdatasets.ai/financial-metrics/"
        f"?ticker={ticker}"
        f"&report_period_lte={report_period}"
        f"&limit={limit}"
        f"&period={period}"
    )
  response = requests.get(url,headers = headers)
  if response.status_code != 200 :
    raise Exception(
        f"Error Fetching the data : {response.status_code} - {response.text}"
    )
  data = response.json()
  financial_metrics = data.get("financial_metrics")
  if financial_metrics:
    raise ValueError("No Financial Metrics returned")
  return financial_metrics


In [ ]:
def search_line_items(ticker:str,line_items:List[str],period:str = "ttm" ,limit : int =1 ) -> List[Dict[str,Any]]:
  """Fetch cash flow statements from the API."""
  headers = {"X-API-KEY": os.environ.get("FINANCIAL_DATASETS_API_KEY")}
  url = "https://api.financialdatasets.ai/financials/search/line-items"
  body = {
      "tickers" : [ticker],
      "line_items":line_items,
      "period":period,
      "limit":limit
  }
  response =  requests.post(url,headers = headers,json=body)
  if response.status_code != 200:
    raise Exception(
        f"Error Fetching data: {r5esponse.status_code} - {response.text}"
    )
    data = response.json()
    search_results = data.get("search_results")
    if not search_results:
      raise ValueError("No search results returned")
    return search_results


In [ ]:
def get_insider_trades(ticker:str,end_date:str,limit:int=5)-> List[Dict[str,Any]]:
  """
    Fetch insider trades for a given ticker and date range.
   """
  headers = {"X-API-KEY": os.environ.get("FINANCIAL_DATASETS_API_KEY")}
  url = (
      f"https://api.financialdatasets.ai/insider-trades/"
      f"?ticker={ticker}"
      f"&filing_date_lte={end_date}"
      f"&limit={limit}"
  )
  response = requests.get(url,headers=headers)
  if response.status_code != 200:
    raise Exception(
        f"Error Fetching data: {response.status_code}  - {response.text}"
    )

  data = response.json()
  insider_trades = data.get("insider_trades")
  if not insider_trades:
    raise ValueError("No Insider Trades returned")
  return insider_trades


In [ ]:
def get_market_cap(ticker:str)->List[Dict[str,Any]]:
  """Fetch market cap from the API."""
  headers = {"X-API-KEY": os.environ.get("FINANCIAL_DATASETS_API_KEY")}
  url = (
      f'https://api.financialdatasets.ai/company/facts'
      f'?ticker={ticker}'
  )
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
      raise Exception(
          f"Error fetching data: {response.status_code} - {response.text}"
      )
  data = response.json()
  company_facts = data.get("company_facts")
  if not company_facts:
      raise ValueError("No company facts returned")
  return company_facts.get('market_cap')



In [ ]:
def get_prices(ticker:str,start_date:str,end_date:str)->List[Dict[str,Any]]:
  """Fetch prices data from the API."""
  headers = {"X-API-KEY": os.environ.get("FINANCIAL_DATASETS_API_KEY")}
  url = (
      f"https://api.financialdatasets.ai/prices/"
      f"?ticker={ticker}"
      f"&interval=day"
      f"&interval_multiplier=1"
      f"&start_date={start_date}"
      f"&end_date={end_date}"
  )
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
      raise Exception(
          f"Error fetching data: {response.status_code} - {response.text}"
      )
  data = response.json()
  prices = data.get("prices")
  if not prices:
      raise ValueError("No price data returned")
  return prices

In [ ]:
def prices_to_df(prices: List[Dict[str, Any]]) -> pd.DataFrame:
    """Convert prices to a DataFrame."""
    df = pd.DataFrame(prices)
    df["Date"] = pd.to_datetime(df["time"])
    df.set_index("Date", inplace=True)
    numeric_cols = ["open", "close", "high", "low", "volume"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df.sort_index(inplace=True)
    return df
def get_price_data(
    ticker: str,
    start_date: str,
    end_date: str
) -> pd.DataFrame:
    prices = get_prices(ticker, start_date, end_date)
    return prices_to_df(prices)

## Agents

### Fundamentals Agent

In [ ]:
from langchain_core.messages import HumanMessage
import json
def fundamentals_agent(state: AgentState):
    """Analyzes fundamental data and generates trading signals."""
    data = state["data"]
    end_date = data["end_date"]


    financial_metrics = get_financial_metrics(
        ticker=data["ticker"],
        report_period=end_date,
        period="ttm",
        limit=1,
    )


    metrics = financial_metrics[0]


    signals = []
    reasoning = {}


    return_on_equity = metrics.get("return_on_equity")
    net_margin = metrics.get("net_margin")
    operating_margin = metrics.get("operating_margin")

    thresholds = [
        (return_on_equity, 0.15),
        (net_margin, 0.20),
        (operating_margin, 0.15),
    ]
    profitability_score = sum(
        metric is not None and metric > threshold for metric, threshold in thresholds
    )

    signals.append(
        "bullish"
        if profitability_score >= 2
        else "bearish" if profitability_score == 0 else "neutral"
    )
    reasoning["profitability_signal"] = {
        "signal": signals[0],
        "details": (
            f"ROE: {metrics['return_on_equity']:.2%}"
            if metrics["return_on_equity"]
            else "ROE: N/A"
        )
        + ", "
        + (
            f"Net Margin: {metrics['net_margin']:.2%}"
            if metrics["net_margin"]
            else "Net Margin: N/A"
        )
        + ", "
        + (
            f"Op Margin: {metrics['operating_margin']:.2%}"
            if metrics["operating_margin"]
            else "Op Margin: N/A"
        ),
    }


    revenue_growth = metrics.get("revenue_growth")
    earnings_growth = metrics.get("earnings_growth")
    book_value_growth = metrics.get("book_value_growth")

    thresholds = [
        (revenue_growth, 0.10),
        (earnings_growth, 0.10),
        (book_value_growth, 0.10),
    ]
    growth_score = sum(
        metric is not None and metric > threshold for metric, threshold in thresholds
    )

    signals.append(
        "bullish"
        if growth_score >= 2
        else "bearish" if growth_score == 0 else "neutral"
    )
    reasoning["growth_signal"] = {
        "signal": signals[1],
        "details": (
            f"Revenue Growth: {metrics['revenue_growth']:.2%}"
            if metrics["revenue_growth"]
            else "Revenue Growth: N/A"
        )
        + ", "
        + (
            f"Earnings Growth: {metrics['earnings_growth']:.2%}"
            if metrics["earnings_growth"]
            else "Earnings Growth: N/A"
        ),
    }


    current_ratio = metrics.get("current_ratio")
    debt_to_equity = metrics.get("debt_to_equity")
    free_cash_flow_per_share = metrics.get("free_cash_flow_per_share")
    earnings_per_share = metrics.get("earnings_per_share")

    health_score = 0
    if current_ratio and current_ratio > 1.5:
        health_score += 1
    if debt_to_equity and debt_to_equity < 0.5:  #To know more about --> Conservative debt levels concept
        health_score += 1
    if (
        free_cash_flow_per_share
        and earnings_per_share
        and free_cash_flow_per_share > earnings_per_share * 0.8
    ):
        health_score += 1

    signals.append(
        "bullish"
        if health_score >= 2
        else "bearish" if health_score == 0 else "neutral"
    )
    reasoning["financial_health_signal"] = {
        "signal": signals[2],
        "details": (
            f"Current Ratio: {metrics['current_ratio']:.2f}"
            if metrics["current_ratio"]
            else "Current Ratio: N/A"
        )
        + ", "
        + (
            f"D/E: {metrics['debt_to_equity']:.2f}"
            if metrics["debt_to_equity"]
            else "D/E: N/A"
        ),
    }


    pe_ratio = metrics.get("price_to_earnings_ratio")
    pb_ratio = metrics.get("price_to_book_ratio")
    ps_ratio = metrics.get("price_to_sales_ratio")

    thresholds = [
        (pe_ratio, 25),
        (pb_ratio, 3),
        (ps_ratio, 5),
    ]
    price_ratio_score = sum(
        metric is not None and metric > threshold for metric, threshold in thresholds
    )

    signals.append(
        "bullish"
        if price_ratio_score >= 2
        else "bearish" if price_ratio_score == 0 else "neutral"
    )
    reasoning["price_ratios_signal"] = {
        "signal": signals[3],
        "details": (f"P/E: {pe_ratio:.2f}" if pe_ratio else "P/E: N/A")
        + ", "
        + (f"P/B: {pb_ratio:.2f}" if pb_ratio else "P/B: N/A")
        + ", "
        + (f"P/S: {ps_ratio:.2f}" if ps_ratio else "P/S: N/A"),
    }


    bullish_signals = signals.count("bullish")
    bearish_signals = signals.count("bearish")

    if bullish_signals > bearish_signals:
        overall_signal = "bullish"
    elif bearish_signals > bullish_signals:
        overall_signal = "bearish"
    else:
        overall_signal = "neutral"


    total_signals = len(signals)
    confidence = round(max(bullish_signals, bearish_signals) / total_signals, 2) * 100

    message_content = {
        "signal": overall_signal,
        "confidence": confidence,
        "reasoning": reasoning,
    }


    message = HumanMessage(
        content=json.dumps(message_content),
        name="fundamentals_agent",
    )


    if state["metadata"]["show_reasoning"]:
        show_agent_reasoning(message_content, "Fundamental Analysis Agent")


    state["data"]["analyst_signals"]["fundamentals_agent"] = {
        "signal": overall_signal,
        "confidence": confidence,
        "reasoning": reasoning,
    }

    return {
        "messages": [message],
        "data": data,
    }


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import  ChatPromptTemplate
from langchain_community.chat_models import ChatVertexAI
def portfolio_management_agent(state:AgentState):
  template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a portfolio manager making final trading decisions.
            Your job is to make a trading decision based on the team's analysis.

            Provide the following in your output:
            - "action": "buy" | "sell" | "hold",
            - "quantity": <positive integer>
            - "confidence": <float between 0 and 1>
            - "reasoning": <concise explanation of the decision including how you weighted the signals>

            Trading Rules:
            - Only buy if you have available cash
            - Only sell if you have shares to sell
            - Quantity must be ≤ current position for sells
            - Quantity must be ≤ max_position_size from risk management""",
        ),
        (
            "human",
            """Based on the team's analysis below, make your trading decision.

            Technical Analysis Trading Signal: {technical_signal}
            Fundamental Analysis Trading Signal: {fundamentals_signal}
            Sentiment Analysis Trading Signal: {sentiment_signal}
            Valuation Analysis Trading Signal: {valuation_signal}
            Risk Management Position Limit: {max_position_size}
            Here is the current portfolio:
            Portfolio:
            Cash: {portfolio_cash}
            Current Position: {portfolio_stock} shares

            Only include the action, quantity, reasoning, and confidence in your output as JSON.  Do not include any JSON markdown.

            Remember, the action must be either buy, sell, or hold.
            You can only buy if you have available cash.
            You can only sell if you have shares in the portfolio to sell.
            """,
        ),
    ]
  )
  portfolio = state["data"]["portfolio"]
  analyst_signals = state["data"]["analyst_signals"]

  prompt = template.invoke(
      {
        "technical_signal":analyst_signals["technical_analyst_agent"]["signal"],
        "funadmentals_signal":analyst_signals["fundamentals_agent"]["signal"],
        "sentiment_signal":analyst_signal["sentiment_agent"]

      }
  )
